# 智能体框架

## 基本功能

### call函数

In [2]:
from textlong.agent import FakeLLM
from textlong.io import log

log(FakeLLM(), "hi")

[INFO] FakeLLM: [{'role': 'user', 'content': 'hi'}]
这是一个模拟调用!



'这是一个模拟调用!'

### async_call 函数

In [3]:
from textlong.agent import FakeLLM
from textlong.io import alog

await alog(FakeLLM(), "hi", sleep=1)

[INFO] FakeLLM: [{'role': 'user', 'content': 'hi'}]
这是一个模拟调用!



'这是一个模拟调用!'

### 线程池

In [3]:
FakeLLM.monitor_executors()

{'FAKE_LLM': {'max_workers': 5, 'used_workers': 1, 'waiting_threads': 0}}

## 调用大模型

### 对话

In [4]:
from textlong.agent import ChatQwen
from textlong.io import log

qwen = ChatQwen()
log(qwen, "你能帮我写一首关于兔子做梦的四句儿歌?")

小白兔，梦中游，  
跳过彩虹，笑悠悠。  
月亮船，云中荡，  
梦里胡萝卜，甜如糖。



'小白兔，梦中游，  \n跳过彩虹，笑悠悠。  \n月亮船，云中荡，  \n梦里胡萝卜，甜如糖。'

In [7]:
qwen.memory

[{'role': 'user', 'content': '你能帮我写一首关于兔子做梦的四句儿歌?'},
 {'role': 'assistant',
  'content': '小白兔，梦中跳，  \n胡萝卜，满天飘。  \n月亮船，带它逛，  \n醒来笑，梦真妙。'}]

### 使用系统提示语

In [5]:
from textlong.agent import ChatQwen
from textlong.io import log

qwen = ChatQwen(prompt="你是一个专门写儿歌的作家，请根据我的提示写作。")
log(qwen, "来一首关于兔子的，四句")

小兔乖乖，耳朵竖起来，
蹦蹦跳跳，草地作舞台。
爱吃萝卜和青菜，生活乐无边，
月光下嬉戏，梦里笑开怀。



'小兔乖乖，耳朵竖起来，\n蹦蹦跳跳，草地作舞台。\n爱吃萝卜和青菜，生活乐无边，\n月光下嬉戏，梦里笑开怀。'

In [2]:
qwen.memory

[{'role': 'system', 'content': '你是一个专门写儿歌的作家，请根据我的提示写作。'},
 {'role': 'user', 'content': '来一首关于兔子的，四句'},
 {'role': 'assistant',
  'content': '小白兔，蹦蹦跳，  \n耳朵长，尾巴小。  \n爱吃萝卜和青菜，  \n快乐生活在林草。'}]

In [3]:
log(qwen, "换成两条小鱼")

两条小鱼，水中游，  
摇摇尾巴，点点头。  
相互追逐，笑哈哈，  
海底世界，真奇妙。



'两条小鱼，水中游，  \n摇摇尾巴，点点头。  \n相互追逐，笑哈哈，  \n海底世界，真奇妙。'

In [4]:
qwen.memory

[{'role': 'system', 'content': '你是一个专门写儿歌的作家，请根据我的提示写作。'},
 {'role': 'user', 'content': '来一首关于兔子的，四句'},
 {'role': 'assistant',
  'content': '小白兔，蹦蹦跳，  \n耳朵长，尾巴小。  \n爱吃萝卜和青菜，  \n快乐生活在林草。'},
 {'role': 'user', 'content': '换成两条小鱼'},
 {'role': 'assistant',
  'content': '两条小鱼，水中游，  \n摇摇尾巴，点点头。  \n相互追逐，笑哈哈，  \n海底世界，真奇妙。'}]

### 返回工具提示

In [1]:
from textlong.tools import tool, convert_to_openai_tool
from textlong.agent import ChatQwen
from textlong.io import log
import json

@tool
def get_current_weather(location: str=None):
    """获取城市的天气情况"""
    return f"{location}今天是晴天。 "

In [2]:
log(ChatQwen(), "今天广州天气如何啊", tools=[convert_to_openai_tool(get_current_weather)])
# log(ChatQwen(), "今天广州天气如何啊", tools=[convert_to_openai_tool(get_current_weather)], toolkits=[get_current_weather])

[TOOLS_CALL_CHUNK] {"index": 0, "id": "call_1ae219216a9b4c78a7423c", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_FINAL] {"0": {"index": 0, "id": "call_1ae219216a9b4c78a7423c", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}


'{"0": {"index": 0, "id": "call_1ae219216a9b4c78a7423c", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\\"location\\": \\"广州\\"}"}}}'

### 执行工具回调

In [3]:
qwen = ChatQwen(
    tools=[convert_to_openai_tool(get_current_weather)],
    toolkits=[get_current_weather]
)

log(qwen, "今天广州可以晒被子吗？")

[TOOLS_CALL_CHUNK] {"index": 0, "id": "call_57a4a0292bba4ae8aaa996", "type": "function", "function": {"name": "get_current_weather", "arguments": ""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "{\"location\": \""}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {"arguments": "广州\"}"}}
[TOOLS_CALL_CHUNK] {"index": 0, "id": "", "type": "function", "function": {}}
[TOOLS_CALL_FINAL] {"0": {"index": 0, "id": "call_57a4a0292bba4ae8aaa996", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\"location\": \"广州\"}"}}}
广州今天是晴天。 

今天广州是晴天，非常适合晒被子。可以好好利用这晴朗的天气给被子来个日光浴哦！



'{"0": {"index": 0, "id": "call_57a4a0292bba4ae8aaa996", "type": "function", "function": {"name": "get_current_weather", "arguments": "{\\"location\\": \\"广州\\"}"}}}今天广州是晴天，非常适合晒被子。可以好好利用这晴朗的天气给被子来个日光浴哦！'

## 管道

### 简单管道 [Chat, Chat, Chat]

In [4]:
from textlong.agent import ChatQwen, Pipe
from textlong.io import log

pipe = Pipe(
    ChatQwen(prompt="我是一个儿童作家，擅长写儿歌。"),
    ChatQwen(prompt="请你帮我评价文章特色，两句话即可"),
    ChatQwen(prompt="请针对我的写作成果打一个分数，给出一句话的打分点，最终给出1分至5分")
)

log(pipe, "你能帮我写一首关于兔子做梦的？四句即可。")

[INFO] >>> Node 1: 我是一个儿童作家，擅长写儿歌。
小兔梦中跳月亮，云朵枕头软又香。
胡萝卜城堡舞会忙，醒来嘴角笑弯弯。

[INFO] >>> Node 2: 请你帮我评价文章特色，两句话即可
这篇文章特色在于其丰富的想象力与温馨的氛围，将孩子般的纯真梦想和奇妙幻想融合得恰到好处，语言生动活泼，充满了诗意与童趣。

[INFO] >>> Node 3: 请针对我的写作成果打一个分数，给出一句话的打分点，最终给出1分至5分
4分。打分点：文章成功营造了独特的想象力和温馨氛围，语言生动，富有诗意与童趣，但仍有提升空间，如在情节连贯性或深度探索上可以进一步加强。



'4分。打分点：文章成功营造了独特的想象力和温馨氛围，语言生动，富有诗意与童趣，但仍有提升空间，如在情节连贯性或深度探索上可以进一步加强。'

In [2]:
pipe.to_runnables[0]['runnable'].memory

[{'role': 'system', 'content': '我将告诉你我的写作成果，你负责帮我评价文章特色，两句话即可'},
 {'role': 'user',
  'content': '小兔梦中跳，月光下舞蹈，\n胡萝卜乐园，梦里吃个饱。\n云朵当被窝，星河伴入眠，\n醒来笑眯眯，是梦也是真。'},
 {'role': 'assistant',
  'content': '你的文章充满了童趣与想象力，字句间跳跃着温馨和梦幻，构建了一个让读者向往的童话世界。通过小兔的梦境，展现了孩童般纯真的快乐与满足，富有诗意，令人会心一笑。'}]

### 简单管道 [Template, Chat, Chat]

In [5]:
from textlong.agent import ChatQwen, Pipe, Template
from textlong.io import log, alog

In [6]:
t = Template(template="IDEA")
log(t, {"task": "请你帮我写一首4行儿歌"})
t.output

[INFO] 你是强大的写作助手。

你必须遵循以下约束来完成任务:
1. 直接输出你的结果，不要评论，不要啰嗦
2. 使用markdown格式输出

**你的任务是:**
请你帮我写一首4行儿歌



'请开始'

In [7]:
pipe = Pipe(
    Template(template="IDEA"),
    ChatQwen(),
    ChatQwen(prompt="请帮我对作品进行评价，找出三个缺点，一句话总结。")
)

log(pipe, {"task": "请你帮我写一首4行儿歌"})

[INFO] >>> Node 1: Template
[INFO] 你是强大的写作助手。

你必须遵循以下约束来完成任务:
1. 直接输出你的结果，不要评论，不要啰嗦
2. 使用markdown格式输出

**你的任务是:**
请你帮我写一首4行儿歌

[INFO] >>> Node 2: ChatQwen
小蜜蜂，嗡嗡嗡  
飞到花丛中  
采花蜜，忙不停  
甜甜蜜带回家

[INFO] >>> Node 3: 请帮我对作品进行评价，找出三个缺点，一句话总结。
1. **内容单一**：诗歌内容围绕蜜蜂采蜜的自然行为，较为简单，缺乏深层次探索或情感波动。
2. **句式重复**：每句以相似的音效词“嗡嗡嗡”开头，虽然增加了节奏感，但也显得创意不足，模式化。
3. **意象缺乏变化**：全诗仅描绘了蜜蜂与花丛的场景，没有展开更多自然环境或蜜蜂生活的其他方面，导致画面感较单一。

**一句话总结**：这首诗简洁明快地捕捉了小蜜蜂勤劳采蜜的生动场景，但因内容、句式及意象上的单一，略显缺乏丰富性和深度。



'1. **内容单一**：诗歌内容围绕蜜蜂采蜜的自然行为，较为简单，缺乏深层次探索或情感波动。\n2. **句式重复**：每句以相似的音效词“嗡嗡嗡”开头，虽然增加了节奏感，但也显得创意不足，模式化。\n3. **意象缺乏变化**：全诗仅描绘了蜜蜂与花丛的场景，没有展开更多自然环境或蜜蜂生活的其他方面，导致画面感较单一。\n\n**一句话总结**：这首诗简洁明快地捕捉了小蜜蜂勤劳采蜜的生动场景，但因内容、句式及意象上的单一，略显缺乏丰富性和深度。'

In [8]:
pipe.memory

[{'role': 'user', 'content': '节点 <0> 正在处理任务...'},
 {'role': 'assistant', 'content': '请开始'},
 {'role': 'user', 'content': '节点 <1> 正在处理任务...'},
 {'role': 'assistant', 'content': '小蜜蜂，嗡嗡嗡  \n飞到花丛中  \n采花蜜，忙不停  \n甜甜蜜带回家'},
 {'role': 'user', 'content': '节点 <2> 正在处理任务...'},
 {'role': 'assistant',
  'content': '1. **内容单一**：诗歌内容围绕蜜蜂采蜜的自然行为，较为简单，缺乏深层次探索或情感波动。\n2. **句式重复**：每句以相似的音效词“嗡嗡嗡”开头，虽然增加了节奏感，但也显得创意不足，模式化。\n3. **意象缺乏变化**：全诗仅描绘了蜜蜂与花丛的场景，没有展开更多自然环境或蜜蜂生活的其他方面，导致画面感较单一。\n\n**一句话总结**：这首诗简洁明快地捕捉了小蜜蜂勤劳采蜜的生动场景，但因内容、句式及意象上的单一，略显缺乏丰富性和深度。'}]

In [10]:
pipe.runnables[0].memory

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n请你帮我写一首4行儿歌\n'},
 {'role': 'user', 'content': '请开始'}]

In [11]:
pipe.runnables[1].memory

[{'role': 'system',
  'content': '你是强大的写作助手。\n\n你必须遵循以下约束来完成任务:\n1. 直接输出你的结果，不要评论，不要啰嗦\n2. 使用markdown格式输出\n\n**你的任务是:**\n请你帮我写一首4行儿歌\n'},
 {'role': 'user', 'content': '请开始'},
 {'role': 'assistant', 'content': '小蜜蜂，嗡嗡嗡  \n飞到花丛中  \n采花蜜，忙不停  \n甜甜蜜带回家'}]

In [12]:
log(pipe.runnables[1], "找出三个优点吧")

1. **寓教于乐**：这首儿歌通过小蜜蜂采蜜的场景，不仅让孩子们感受到自然界的美妙，还能教育他们勤劳与付出的价值。
2. **朗朗上口**：使用了重复的音节如“嗡嗡嗡”，以及简单易懂的词汇，使得儿歌节奏感强，便于儿童记忆和吟唱。
3. **启发想象**：通过描述小蜜蜂的活动，激发孩子们对大自然的好奇心和探索欲，鼓励他们想象自己置身于多彩的花丛之中。



'1. **寓教于乐**：这首儿歌通过小蜜蜂采蜜的场景，不仅让孩子们感受到自然界的美妙，还能教育他们勤劳与付出的价值。\n2. **朗朗上口**：使用了重复的音节如“嗡嗡嗡”，以及简单易懂的词汇，使得儿歌节奏感强，便于儿童记忆和吟唱。\n3. **启发想象**：通过描述小蜜蜂的活动，激发孩子们对大自然的好奇心和探索欲，鼓励他们想象自己置身于多彩的花丛之中。'

In [ ]:
Team([idea, outline, from_outline])